## Introduction to ARIA Python library

This example shows the essential steps needed to connect to a robot or simulator, then continuously receive robot state information and send motion commands.

To run the whole program in Jupyter, select "Run All" from the Cell menu. The output from each cell is shown below it.  When running, a cell will show `[*]` next to it.   To run each cell, use the down arrow key to select each cell, and press `Ctrl-Enter` to run the chunk.  Note that not all cells generate visible output. To edit a cell, double click on it.  To reset the notebook, choose "Restart & Clear All Output" from the Kernel menu, or click the Restart button in the toolbar and choose Cell, All Output, Clear to clear old output.

To use the MobileSim simulator, simply run MobileSim before evaluating the code in this notebook.  To connect to an AmigoBot, Pioneer3 or other robot with a wifi interface, make sure you are on the same network as the wifi interface.  (Otherwise ARIA will attempt to connect through a serial port connection as configured via command-line options from sys.argv or given in an ARIAARGS environment variable.)

First, import the `AriaPy` Python module. (`AriaPy` will load the ARIA native library/DLL when loaded.)

In [1]:
from AriaPy import *

Next, connect to the robot.  If any robots with wifi interfaces are found, you can select one to use.  If the MobileSim simulator is running, connect to that.  Otherwise try to connect to a robot via direct serial connection.  (You can also set parameters to configure the connection in `/etc/Aria.args` or by adding them to `sys.argv` before calling `connecToRobot()`)

An `ArRobot` object is returned, which provides the primary interface to communicating with the robot.

In [2]:
robot = Aria.connectToRobot()

Aria: Searching Network for WiBox Interfaces...
Aria: : No WiBoxes found.
Aria: Connecting to robot...
Aria: Connected.


Now we enable the motors and run the robot's [task cycle](file:///usr/local/Aria/docs/index.html#syncRobot).  This is a loop in which communication with the robot or simulator is maintained, data is received, and can call user-added "task" functions within this regular cycle.  This cycle is synchronized to the data updates received from the robot, so tasks will be called for every update.  (When you run this code, you will see the counter value included in the output below increment with each cycle.)

In [3]:
print('Running...')
robot.enableMotors()
robot.runAsync()

Running...


The `robot` object will now receive regular data updates from the robot on a regular 100 milisecond cycle.  This data includes the robot pose, and other data.  Let's define a function that will print the pose each time:

The pose is the robot's own estimated position based on its movements since starting up.  The pose contains a position on a cartesian coordinate system (X, Y), and a rotation or heading (Theta).  The origin of the coordinate system is  `getPose()` returns an `ArPose` object which contains some useful comparison and other functions. You can access the components as `x`, `y`, and `th` members or call `getX()`, `getY()`, and `getTh()`.  Some helpful math functions are included in the `ArMath` namespace as well.

We use `addSensorInterpTask()` to give this function to the `robot` object so it can automatically call it every 100ms cycle immediately after new position and other information updates are received from the robot.

The first argument is the function to call.  The second one is a name for this task. You can
choose any name you want.  

In [4]:
def printRobotPos():
  print('pose (%d) -- %s' % (robot.getCounter(), robot.getPose()))

robot.addSensorInterpTask(printRobotPos, 'PrintData')

True

Let's also define a function to generate the motion we want the robot to perform. `setVel()` sets desired translational velocity and `setRotVel()` sets desired rotational velocity.  Together, these two components result in a curved trajectory.  

We will use `addUserTask()` this time.  These tasks are called near the end of the robot cycle.


In [ ]:
def driveRobot():
  print('drive (%d)' % (robot.getCounter()))
  robot.setVel(300)
  robot.setRotVel(20)

robot.addUserTask(driveRobot, 'DriveRobot')

If you want to change a task function, you must also replace the task in `robot`.  
Use `replaceSensorInterpTask()` and `replaceUserTask()` to do so, making sure
that the name you give is the same as before. (If you don't plan on ever replacing or
removing the task, you can omit the name, and AriaPy will choose a unique name based on
some internal Python function information.)

In [ ]:
def driveRobot():
    print 'new drive function (%d)' % (robot.getCounter())
    robot.setVel(500)
    roobt.setRotVel(35)

robot.replaceUserTask(driveRobot, 'DriveRobot')

This user task will send velocity commands every cycle, but the robot controller will
maintain any velocity command until a new command is received. So if you want to stop
the robot, you must both remove any tasks that are sending commands, and stop the robot:

In [ ]:
robot.removeTask('DriveRobot')
robot.stop()

*Exercize:* Define a new function that sets different velocity values. Then use `replaceTask()` to replace the `'DriveRobot'` task. To do so in Jupyter, insert a new notebook cell, enter the code, and evaluate it. (Or restart the notebook.)
What happens with  a rotational velocity of 0?  What happens with a translational velocity of 0?   What happens with a large translational velocity but a small rotational velocity? 
What happens with a large rotational velocity and a large translational velocity?